In [1]:
from PatchedMeasCal.edge_bfs import CouplingMapGraph
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter

import qiskit
from qiskit.providers.fake_provider import FakeVigo, FakeTokyo

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
import qiskit.tools.jupyter

In [2]:
n_qubits = 5
n_shots = 32000

backend = FakeVigo()

# If you want to use the same total number of shots to prepare each calibration
n_shots_qiskit = n_shots / (2 ** n_qubits) # One calibration circuit per bitstring
n_shots_patch = n_shots / (2 * len(backend.configuration().coupling_map)) # 4 for each calibration, but the coupling map double counts
#backend = FakeTokyo()

## Qiskit
This will take approximately forever to complete for larger devices

In [3]:
qr = qiskit.QuantumRegister(n_qubits)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
t_qc = qiskit.transpile(meas_calibs, backend)
cal_results = qiskit.execute(t_qc, backend, shots=n_shots_qiskit).result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
meas_filter = meas_fitter.filter

## Patched

In [4]:
tpf = TensorPatchFitter(backend, n_shots=n_shots_patch)
tpf.build(verbose=True)

Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [==========>         ] 100.0% 
	Building Edge Patches : [===============>    ] 100.0% 
	Constructing Calibration Circuits : [==========>         ] 100.0% 
	Executing Calibration Circuits
	De-hexing Measurement Results
	Tracing Patched Calibration Results : [==========>         ] 100.0% 
Building Patch Calibrations
	Merging Patches : [============>       ] 100.0% 
Building Measure Fitter
	Building Meas Fitters from Patches : [==========>         ] 100.0% 


## The circuit to test

In [5]:
circ = qiskit.QuantumCircuit(n_qubits, n_qubits)
initial_layout = list(range(n_qubits))

circ.h(0)
for i in range(1, n_qubits):
    circ.cnot(i - 1, i)

circ.measure(initial_layout, initial_layout)
tc = qiskit.transpile(circ, backend=backend, initial_layout=initial_layout, optimization_level=0)
results = qiskit.execute(tc, backend, shots=n_shots, initial_layout=initial_layout, optimization_level=0).result()

In [6]:
bare_res = results.get_counts()
tpf_res, arr = tpf.apply(bare_res)
qiskit_res = meas_filter.apply(bare_res)

In [7]:
def dist(res, n_shots, n_qubits):
    distance = abs(res['0' * n_qubits] - n_shots / 2) + abs(res['1' * n_qubits] - n_shots / 2)
    distance /= n_shots
    return distance

In [8]:
print(dist(bare_res, n_shots, n_qubits))
print(dist(tpf_res, n_shots, n_qubits))
print(dist(qiskit_res, n_shots, n_qubits))

0.18559375
0.11098349510273607
0.037733469097223346


In [9]:
print(bare_res['1' * n_qubits], bare_res['0' * n_qubits])
print(qiskit_res['1' * n_qubits], qiskit_res['0' * n_qubits])
print(tpf_res['1' * n_qubits], tpf_res['0' * n_qubits])

12408 13653
15132.937092188193 15659.59189670066
14803.716984034723 13644.811172677722


In [10]:
sum(qiskit_res.values())

32000.000000003285

In [11]:
sum(tpf_res.values())

31999.999999999996